# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.<br/>
Данные для проекта предоставлены в одном дф - *data.csv*.<br/>

**Задачи**:
- выполнить предобработку данных;
- ответить на вопросы заказчика:
 * Есть ли зависимость между наличием детей и возвратом кредита в срок?
 * Есть ли зависимость между семейным положением и возвратом кредита в срок?
 * Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
 * Как разные цели кредита влияют на его возврат в срок?
 
**Цель проекта** выявить критерии надежности заемщика.

## Изучение данных

In [1]:

import pandas as pd
from pymystem3 import Mystem
m = Mystem()
from collections import Counter
data = pd.read_csv('/datasets/data.csv')
data.info()
data.head(15)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Вывод**

С помощью метода info() видим, что таблица имеет 21524 наблюдения и 12 столбцов. В столбцах корректный тип данных (кроме типа float в столбцах "общий трудовой стаж в днях" и "ежемесячный доход"), и названы в одном стиле, чего не скажешь о самих данных - указаны разным регистром, в столбце "общий трудовой стаж в днях" отрицательные значения.
Также в глаза бросается, что столбцы "общий трудовой стаж в днях" и "ежемесячный доход" имеют пустые значения.

## Предобработка данных

### Обработка пропусков

In [2]:
print('Количество пропущенных значений')
print(data.isna().sum()) #Проверим, сколько пропущенных значений имеют столбцы "общий трудовой стаж в днях" и "ежемесячный доход".
mean_day_employed = data['days_employed'].mean() #считаем среднее арифметическое значение "общий трудовой стаж в днях"
print()
print('Вид занятости клиентов, оформивших кредиты')
print(data['income_type'].value_counts())
print()
print('Количество пропусков в столбце доход клиентов, по виду занятости')

data['days_employed'] = data['days_employed'].abs()
data['days_employed'] = data['days_employed'].fillna(mean_day_employed)
data['total_income'] = data.groupby('income_type').transform(lambda group: group.fillna(group.mean()))['total_income']
print()
print('Количество пропущенных значений после замены')
print(data.isna().sum())
display(data)

Количество пропущенных значений
children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Вид занятости клиентов, оформивших кредиты
сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
предприниматель        2
безработный            2
студент                1
в декрете              1
Name: income_type, dtype: int64

Количество пропусков в столбце доход клиентов, по виду занятости

Количество пропущенных значений после замены
children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dty

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля


Пустые значения в столбце "общий трудовой стаж в днях" заменил на среднее арифметическое.<br/>
Пустые значения в столбце "ежемесячный доход" заменил на групповые средние значения.
Не стал удалять строки с пустыми значениями, т.к. эти данные это 10% от общего количества наблюдений. Избивался от отрицательных значений в стобце "общий трудовой стаж в днях".
Вопрос вызывает категория пенсионеров с медианным ежемесячным доходом, целью кредита и общим трудовым стажем-по таким данным лучше попросить более качественную выгрузку.





### Замена типа данных

In [3]:
#меняем тип данных в столбце "общий трудовой стаж в днях" и "ежемесячный доход"
data[['days_employed','total_income']] = data[['days_employed','total_income']].astype(int) 
data.info()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу


**Вывод**

Изменил тип данных в столбцах "общий трудовой стаж в днях" и "ежемесячный доход" на integer методом astype для более удобного визуального анализа и удобства решения задач по зависимости дохода и вовзрата кредита в срок.

### Обработка дубликатов

In [4]:
#меняем регистр данных в столбцах "образование" и "семейное положение" на нижний
data['education'] = data['education'].str.lower()
data['family_status'] = data['family_status'].str.lower()
#избавляемся от артефактов в столбце "количетсво детей" 
data['children'] = data['children'].replace(-1, 1)
median_children = data['children'].median()
data['children'] = data['children'].replace(20, median_children)
data_children = pd.DataFrame({'Количество заемщиков по количеству детей': data['children'].value_counts()})
data_children


,Количество заемщиков по количеству детей
0,14225
1,4865
2,2055
3,330
4,41
5,9


**Вывод**

Методом values_counts() проверил значения столбцов.
Данные в столбце "образование" и "семейное положение" привел к общему виду - нижнему регистру. После проверки столбца "количество детей" выявилось отрицательное значение (-1) и значение "20". Эти значения заменил методом replaсe() на положительное и медианное значение (т.к. не известно ошибочно введен 0 или 2) соответственно. В столбце "пол" найдено значение "XNA", поскольку сложно сделать предположение о поле человека и данный столбец не участвует в задачах анализа - оставил это значение без изменений.
В качестве вывода привел таблицу количества заемщиков по количеству детей. Из нее видно, что чем больше детей, тем реже берут кредиты. 

### Лемматизация

In [5]:
m_old = Mystem
display(m_old.lemmatize)
display(m.lemmatize)
Counter(m_old.lemmatize(m,",".join(data['purpose'])))

<function pymystem3.mystem.Mystem.lemmatize(self, text)>

<bound method Mystem.lemmatize of <pymystem3.mystem.Mystem object at 0x7fe06589ec10>>

Counter({'покупка': 5912,
         ' ': 33677,
         'жилье': 4473,
         ',': 21524,
         'приобретение': 462,
         'автомобиль': 4315,
         'дополнительный': 909,
         'образование': 4022,
         'сыграть': 774,
         'свадьба': 2348,
         'операция': 2610,
         'с': 2924,
         'на': 2233,
         'проведение': 777,
         'для': 1294,
         'семья': 641,
         'недвижимость': 6367,
         'коммерческий': 1315,
         'жилой': 1233,
         'строительство': 1881,
         'собственный': 635,
         'подержать': 858,
         'свой': 2235,
         'со': 630,
         'заниматься': 908,
         'сделка': 944,
         'получение': 1316,
         'высокий': 1375,
         'подержанный': 110,
         'профильный': 436,
         'сдача': 653,
         'ремонт': 612,
         '\n': 1})

**Вывод**

### Категоризация данных

In [6]:
#лемматизируем столбец "цель получения кредита" и создаем новый столбец с категориями

estate = ['недвижим', 'жиль']
wedding = ['свадьб']
auto = ['автомоб']
education = ['образова']
data['purpose_category'] = 0
def categorizer(list_of_words, category):
    join = '|'.join(list_of_words)
    index = data[data['purpose'].str.lower().str.contains(join)].index.to_list()
    for i in index:
        data.loc[i, 'purpose_category'] = category
    return data
categorizer(estate, 'недвижимость')
categorizer(wedding, 'свадьба')
categorizer(auto, ' авто')
categorizer(education, 'образование')
purpose_data = pd.DataFrame({'Цель получения кредита': data['purpose_category'].value_counts()})
print(purpose_data)
print(purpose_data.sum())

              Цель получения кредита
недвижимость                   10840
 авто                           4315
образование                     4022
свадьба                         2348
Цель получения кредита    21525
dtype: int64


**Вывод**

Создал отдельную таблицу, из которой видно, что основная доля кредитов занимает ипотека, которая более чем на 50 % больше автокредита и кредита на образования и почти в 5 раз больше доли кредитов, оформленных на свадебные торжества. Вполне логическое соотношение, с учетом цен и приоритетов населения. Cумма всех целей получения кредита cходится с количеством наблюдений в исходном датафрейме, значит все цели определены верно.

## Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [7]:
#создаем таблицу зависимости наличия детей и возвратом кредита  в срок, с сортировкой по количеству детей и частотой просрочки
debt_children = pd.DataFrame()
debt_children['count'] = data.groupby('children')['debt'].count()
debt_children['sum'] = data.groupby('children')['debt'].sum()
debt_children['result'] = debt_children['sum'] / debt_children['count']
debt_children['result'] = debt_children['result'].map(lambda x: '{:,.3f}'.format(x))
debt_children.sort_values('result', ascending  = False)


,count,sum,result
children,,,
4,41,4,0.098
2,2055,194,0.094
1,4865,445,0.091
3,330,27,0.082
0,14225,1071,0.075
5,9,0,0.000


**Вывод**

Чаще всего просрочки допускают клиенты с четырьмя и двумя детьми. Самые добросовестные плательщики это клиенты, у которых детей нет.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [8]:
#создаем таблицу зависимости семейного положения и возвратом кредита  в срок,
#с сортировкой по семейному положению и частотой просрочки
debt_family_status = pd.DataFrame()
debt_family_status['count'] = data.groupby('family_status')['debt'].count()
debt_family_status['sum'] = data.groupby('family_status')['debt'].sum()
debt_family_status['result'] = debt_family_status['sum'] / debt_family_status['count']
debt_family_status['result'] = debt_family_status['result'].map(lambda x: '{:,.3f}'.format(x))
debt_family_status.sort_values('result', ascending  = False)

,count,sum,result
family_status,,,
не женат / не замужем,2813,274,0.097
гражданский брак,4177,388,0.093
женат / замужем,12380,931,0.075
в разводе,1195,85,0.071
вдовец / вдова,960,63,0.066


**Вывод**

Чаще всего просрочки допускают клиенты, которые не состоят в браке и состоят в гражданском браке. Самые добросовестные плательщики это клиенты, которые находятся в разводе или вдовцы (вдовы).

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [9]:
#создаем функцию, для категоризации доходов по 50000, которые будут записываться в новый столбец по id ()
def total_income_cat(value):
    if value >= 500000:
        return 10 #единственная категория с условием, остальные будут делится следующим образом - сумма дохода делится на 50000
# и частное (целое число) результата будет являться id категории
    else:
        value = value // 50000
        return value
data['total_income_id'] = data['total_income'].apply(total_income_cat)
data['total_income_id'].value_counts()
debt_total_income = pd.DataFrame()
debt_total_income['count'] = data.groupby('total_income_id')['debt'].count()
debt_total_income['sum'] = data.groupby('total_income_id')['debt'].sum()
debt_total_income['result'] = debt_total_income['sum'] / debt_total_income['count']
debt_total_income['result'] = debt_total_income['result'].map(lambda x: '{:,.3f}'.format(x))
debt_total_income.sort_values('result', ascending  = False)


,count,sum,result
total_income_id,,,
3,5370,473,0.088
2,6117,526,0.086
6,624,51,0.082
1,4091,331,0.081
7,330,24,0.073
4,2762,194,0.070
5,1330,88,0.066
8,196,13,0.066
10,222,14,0.063


**Вывод**

Чаще всего просрочки допускают клиенты, у которых зарплата от 50 до 200 т.р., но так получается, из-за того, что это средняя зарплата по стране и, соответственно, клиентов с такой зарплатой больше. В целом прямой зависимости между доходом и возвратом кредитов в срок нет.

- Как разные цели кредита влияют на его возврат в срок?

In [10]:
total_debt = data['debt'].sum()/ len(data)
#создаем табилцу зависимости цели кредита и возврата кредита в срок
debt_purpose = pd.DataFrame()
debt_purpose['count'] = data.groupby('purpose_category')['debt'].count()
debt_purpose['sum'] = data.groupby('purpose_category')['debt'].sum()
debt_purpose['result'] = debt_purpose['sum'] / debt_purpose['count']
debt_purpose['total_debt'] = debt_purpose['result'] / total_debt
debt_purpose['result'] = debt_purpose['result'].map(lambda x: '{:,.3f}'.format(x))
debt_purpose['total_debt'] = debt_purpose['total_debt'].map(lambda x: '{:,.3f}'.format(x))
debt_purpose.sort_values('result', ascending  = False)

,count,sum,result,total_debt
purpose_category,,,,
авто,4315,403,0.093,1.155
образование,4022,370,0.092,1.137
свадьба,2348,186,0.079,0.979
недвижимость,10840,782,0.072,0.892


**Вывод**

Чаще всего просрочки допускают по автокредитам и кредитам на образование. По ипотечным кредитам просрочки допускаются реже всего.

## Общий вывод

Из приведенного анализа складывается портрет идеального заемщика-вдовец без детей, с зарплатой от 450 000 р до 500 000 р, который планирует оформить кредит на покупку недвижимости.
Выводы кроме шуток:
 1. клиенты с детьми чаще допускают просрочки, т.к. наличие детей всегда подразумевает дополнительные и непредвиденные расходы;
 2. клиенты, которые состоят или состояли в официальном браке более добросовестные плательщики, нежели холостые и те, кто состоит в гражданском браке;
 3. сумма дохода серьезно не влияет на возврат кредита в срок, т.к. сумма одобренного кредита зависит от ежемесячного дохода, что, в свою очереь, адекватно распределяет кредитную нагрузку на клиента;
 4. меньше всего просрочек допускается по ипотечным кредитам. Это зависит от того, что платежи по ипотечным кредитам самые большие и в случае просрочки начисляются пени, которые могут серьезно ухудшить финансовое состояние клиента, а в самом худшем сценарии неоплаты кредита можно вообще лишиться жилья. Эти факторы мотивируют вносить своевременые ежемесячные платежи.